# Web scraping dat z webu
Pomocí následujícího kódu jsem ze stránek elferspot.com získal potřebná data <br>
Hlavním nástrojem je knihovna BeautifulSoup, která dokáže získat údaje z webových stránek <br>
Výsledná data jsem importoval do .csv souboru a následně upravil v excelu

In [6]:
# Import knihoven
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import time

In [8]:
time.sleep(2)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.121 Safari/537.36 AOLShield/109.0.5414.12"
}

In [20]:
# Web scraping
def handle_missing_value(value):
    if value is None or value == "":
        return "#N/A"
    return value

porsche_data = []

# Tento for loop projde všech 94 stránek a jednotlivá auta na webu bazaru a z nich získá potřebná data
for x in range(1, 94):
    r = requests.get(f"https://www.elferspot.com/en/search/page/{x}/", headers=headers)
    soup = bs(r.content, "lxml")
    
    productlist = soup.find_all("a", class_="content-teaser")
    
    for link in productlist:
        href = link.get('href')
        car_url = f"https://www.elferspot.com{href}" if href.startswith('/') else href
        
        car_r = requests.get(car_url, headers=headers)
        car_soup = bs(car_r.content, "lxml")
        
        price_tag = car_soup.find("span", class_="p")
        price = handle_missing_value(price_tag.text.strip() if price_tag else None)

        table_data = {
            "Price": price,
            "Year": "#N/A",
            "Model": "#N/A",
            "Body": "#N/A",
            "Series:": "#N/A",
            "Mileage": "#N/A",
            "Power": "#N/A",
            "Cylinder Capacity": "#N/A",
            "Steering": "#N/A",
            "Transmission": "#N/A",
            "Drive": "#N/A",
            "Fuel": "#N/A",
            "Interior Material": "#N/A",
            "Interior Color": "#N/A",
            "Exterior Color": "#N/A",
            "Condition": "#N/A",
            "New / Used": "#N/A",
            "Ready to Drive": "#N/A",
            "Location": "#N/A"
        }

        tables = car_soup.find_all("table", class_="fahrzeugdaten")
        
        if tables:
            for table in tables:
                rows = table.find_all("tr")
                
                for row in rows:
                    label = row.find("td", class_="label")
                    content = row.find("td", class_="content")
                    
                    if label and content:
                        label_text = label.get_text(strip=True)
                        link = content.find("a")
                        if link:
                            content_text = link.get_text(strip=True)
                        else:
                            content_text = content.get_text(strip=True)

                        if label_text == "Year of construction:":
                            table_data["Year"] = handle_missing_value(content_text)
                        elif label_text == "Model:":
                            table_data["Model"] = handle_missing_value(content_text)
                        elif label_text == "Body:":
                            table_data["Body"] = handle_missing_value(content_text)
                        elif label_text == "Series:":
                            table_data["Series:"] = handle_missing_value(content_text)
                        elif label_text == "Mileage:":
                            table_data["Mileage"] = handle_missing_value(content_text)
                        elif label_text == "Power:":
                            table_data["Power"] = handle_missing_value(content_text)
                        elif label_text == "Cylinder capacity:":
                            table_data["Cylinder Capacity"] = handle_missing_value(content_text)
                        elif label_text == "Steering:":
                            table_data["Steering"] = handle_missing_value(content_text)
                        elif label_text == "Transmission:":
                            table_data["Transmission"] = handle_missing_value(content_text)
                        elif label_text == "Drive:":
                            table_data["Drive"] = handle_missing_value(content_text)
                        elif label_text == "Fuel:":
                            table_data["Fuel"] = handle_missing_value(content_text)
                        elif label_text == "Interior material:":
                            table_data["Interior Material"] = handle_missing_value(content_text)
                        elif label_text == "Interior color:":
                            table_data["Interior Color"] = handle_missing_value(content_text)
                        elif label_text == "Exterior color:":
                            table_data["Exterior Color"] = handle_missing_value(content_text)
                        elif label_text == "Condition:":
                            table_data["Condition"] = handle_missing_value(content_text)
                        elif label_text == "New / used:":
                            table_data["New / Used"] = handle_missing_value(content_text)
                        elif label_text == "Ready to drive:":
                            table_data["Ready to Drive"] = handle_missing_value(content_text)
                        elif label_text == "Car location:":
                            table_data["Location"] = handle_missing_value(content_text)

        porsche_data.append(table_data)

In [ ]:
# Vytvoření DataFrame a následná transformace do .csv souboru
df = pd.DataFrame(porsche_data)
df.to_csv("porsche_data.csv", index=False)